# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Input File (CSV)
input_data_file = 'output_data/clean_cities.csv'

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
clean_city_df = pd.read_csv(input_data_file)
clean_city_df

,City,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloudiness,Country,Date
0,Iqaluit,-68.51,63.75,55.40,87,16.11,90,CA,1596674195
1,Kapaa,-159.32,22.08,82.40,74,10.29,90,US,1596674195
2,Saint-Philippe,55.77,-21.36,62.60,63,6.93,40,RE,1596673953
3,Rikitea,-134.97,-23.12,70.68,74,9.80,17,PF,1596673958
4,San Quintín,-115.95,30.48,70.43,68,19.19,0,MX,1596674197
...,...,...,...,...,...,...,...,...,...
554,Kawakawa,174.07,-35.38,63.00,3,7.00,100,NZ,1596674408
555,Vuktyl,57.31,63.86,59.86,82,9.08,92,RU,1596674409
556,Kayerkan,87.76,69.35,57.20,82,4.47,0,RU,1596674409
557,Khudumelapye,24.75,-23.88,52.18,38,6.13,0,BW,1596674409


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
long_lat = clean_city_df[['Lng','Lat']]
coordinates = long_lat.to_records(index=False)
humidity = clean_city_df['Humidity']

med_long = clean_city_df['Lng'].median()
med_lat = clean_city_df['Lat'].median()
max_humid = clean_city_df['Humidity'].max()

figure_layout = {
    'width': '650px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
    }

gmaps.configure(api_key=g_key)
fig = gmaps.figure(layout=figure_layout, center=(med_long,med_lat),
                   zoom_level=1.5, display_toolbar=False)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [4]:
heat_layer = gmaps.heatmap_layer(locations=coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=max_humid,
                                 point_radius=1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_temps_rng = np.arange(60, 77, 1)
ideal_humid_rng = np.arange(60, 70, 1)
ideal_wind_speed_rng = np.arange(2, 7, 1)

not_ideal_weather_df = clean_city_df.loc[(~clean_city_df['Max Temp'].isin(ideal_temps_rng)) &
                                         (~clean_city_df['Humidity'].isin(ideal_humid_rng)) &
                                         (~clean_city_df['Wind Speed'].isin(ideal_wind_speed_rng))]

ideal_weather_cities = (clean_city_df.drop(index=not_ideal_weather_df.index)).dropna().reset_index(drop=True)
ideal_weather_cities

,City,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloudiness,Country,Date
0,Saint-Philippe,55.77,-21.36,62.60,63,6.93,40,RE,1596673953
1,San Quintín,-115.95,30.48,70.43,68,19.19,0,MX,1596674197
2,Eyl,49.82,7.98,79.03,63,23.35,96,SO,1596674198
3,Port Lincoln,135.87,-34.73,51.39,63,20.69,100,AU,1596674129
4,Provideniya,-173.30,64.38,55.40,66,13.42,75,RU,1596674200
...,...,...,...,...,...,...,...,...,...
116,Serebryanyy Bor,124.80,56.80,70.00,49,2.06,7,RU,1596674405
117,City of San Pedro,121.02,14.35,87.01,72,4.00,99,PH,1596674406
118,Constitución,-72.42,-35.33,48.96,67,8.99,0,CL,1596674407
119,Kawakawa,174.07,-35.38,63.00,3,7.00,100,NZ,1596674408


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_weather_cities
hotel_df['Hotel Name'] = ''
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloudiness,Country,Date,Hotel Name
0,Saint-Philippe,55.77,-21.36,62.60,63,6.93,40,RE,1596673953,
1,San Quintín,-115.95,30.48,70.43,68,19.19,0,MX,1596674197,
2,Eyl,49.82,7.98,79.03,63,23.35,96,SO,1596674198,
3,Port Lincoln,135.87,-34.73,51.39,63,20.69,100,AU,1596674129,
4,Provideniya,-173.30,64.38,55.40,66,13.42,75,RU,1596674200,
...,...,...,...,...,...,...,...,...,...,...
116,Serebryanyy Bor,124.80,56.80,70.00,49,2.06,7,RU,1596674405,
117,City of San Pedro,121.02,14.35,87.01,72,4.00,99,PH,1596674406,
118,Constitución,-72.42,-35.33,48.96,67,8.99,0,CL,1596674407,
119,Kawakawa,174.07,-35.38,63.00,3,7.00,100,NZ,1596674408,


In [ ]:
col_len = len(hotel_df.columns)
city_coord = hotel_df[['Lng','Lat']]
hotel_info = {}

# # base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for i in range(len(city_coord)):
    try:
        # geocoordinates
        target_coordinates = f'{city_coord.iloc[i,0]},{city_coord.iloc[i,1]}'
        target_radius = 5000
        target_type = "hotel"
        
        # set up a parameters dictionary
        params = {
            'location': target_coordinates,
            'radius': target_radius,
            'type': target_type,
            'key': g_key
        }
    
        # run a request using our params dictionary
        response = requests.get(base_url, params=params).json()
#         print(json.dumps(response, indent=4))
        
        hotel_info.update({'Lat': response['results'][0]['geometry']['location']['lat'],
                           'Lng': response['results'][0]['geometry']['location']['lng']})
    
        hotel_df.iloc[i,9] = hotel_info.update
        
    except:
        print(f'Error.')

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
